In [ ]:
#!/usr/bin/env python3
"""
Task 1: Create data splits (random sampling):
 - Validation Set: 1000 sentences
 - Test Set: 1000 sentences
 - Training Set: remaining sentences

Reads 'sentence_tokens' (supports .json, .pkl, .txt).
Outputs: train.pkl, val.pkl, test.pkl (pickle lists of token lists) and small summary.
"""

import os, json, pickle, random
from pathlib import Path

INPUT = "sentence_tokens"   # will try .json .pkl .txt or bare file
VAL_SIZE = 1000
TEST_SIZE = 1000
SEED = 42

def load_sentences(base):
    candidates = [base, base + ".json", base + ".pkl", base + ".txt"]
    for f in candidates:
        if not os.path.exists(f):
            continue
        if f.endswith(".pkl"):
            with open(f, "rb") as fh:
                data = pickle.load(fh)
                if isinstance(data, list):
                    return data
        elif f.endswith(".txt"):
            with open(f, "r", encoding="utf8") as fh:
                lines = [line.strip().split() for line in fh if line.strip()]
                return lines
        else:
            # try JSON or bare
            try:
                with open(f, "r", encoding="utf8") as fh:
                    data = json.load(fh)
                    if isinstance(data, list):
                        return data
            except Exception:
                # treat as plain text fallback
                with open(f, "r", encoding="utf8") as fh:
                    lines = [line.strip().split() for line in fh if line.strip()]
                    return lines
    raise FileNotFoundError("Couldn't find sentence_tokens(.json/.pkl/.txt)")

def save_pickle(obj, fn):
    with open(fn, "wb") as fh:
        pickle.dump(obj, fh)

def main():
    sentences = load_sentences(INPUT)
    print(f"Loaded {len(sentences)} sentences.")
    random.seed(SEED)
    random.shuffle(sentences)
    if len(sentences) < VAL_SIZE + TEST_SIZE:
        raise ValueError("Not enough sentences to create the requested splits.")
    val = sentences[:VAL_SIZE]
    test = sentences[VAL_SIZE:VAL_SIZE+TEST_SIZE]
    train = sentences[VAL_SIZE+TEST_SIZE:]
    save_pickle(train, "train.pkl")
    save_pickle(val, "val.pkl")
    save_pickle(test, "test.pkl")
    print("Splits written: train.pkl ({}), val.pkl ({}), test.pkl ({})".format(len(train), len(val), len(test)))
    # also write simple summaries
    with open("split_summary.txt", "w", encoding="utf8") as fh:
        fh.write(f"Total sentences: {len(sentences)}\n")
        fh.write(f"Train: {len(train)}\nVal: {len(val)}\nTest: {len(test)}\n")
    print("Done.")

if __name__ == "__main__":
    main()


Loaded 144154 sentences.
Splits written: train.pkl (142154), val.pkl (1000), test.pkl (1000)
Done.


In [ ]:
#!/usr/bin/env python3
"""
Task 2: Build unigram/bigram/trigram/quadrigram counts on TRAIN and compute Good-Turing
adjusted joint probabilities for each n (1..4).

Input: train.pkl (produced by task1)
Outputs:
 - counts_n{1..4}.pkl  (Counter of n-grams)
 - gt_joint_n{1..4}.pkl (dict: seen ngram -> probability)
 - diagnostics_n{1..4}.json (Nc, N1, N_distinct, total_tokens_observed)
"""

import pickle, math, json
from collections import Counter, defaultdict
from pathlib import Path

TRAIN_PICKLE = "train.pkl"
MAX_N = 4
SPECIAL_START = "<s>"
SPECIAL_END = "</s>"

def load_pickle(fn):
    with open(fn, "rb") as fh:
        return pickle.load(fh)

def ngrams_from_sentence(tokens, n):
    padded = [SPECIAL_START] * (n-1) + tokens + [SPECIAL_END]
    return [tuple(padded[i:i+n]) for i in range(len(padded)-n+1)]

def build_counts(sentences, n):
    c = Counter()
    total = 0
    for s in sentences:
        ngs = ngrams_from_sentence(s, n)
        c.update(ngs)
        total += len(ngs)
    return c, total

def compute_Nc(counter):
    freq_of_freq = Counter(counter.values())
    return dict(freq_of_freq)

def good_turing_counts(counter, V_power_n=None):
    """
    Compute C* per seen n-gram using c* = (c+1) * N_{c+1} / N_c when possible.
    Also compute Punseen per unseen n-gram using formula from prompt if V_power_n provided.
    Returns: C_star_dict (ngram->c_star), Punseen_per_unseen (float or None), diagnostics
    """
    Nc = compute_Nc(counter)
    N_distinct = len(counter)
    N1 = Nc.get(1, 0)
    max_c = max(counter.values()) if counter else 0
    C_star = {}
    for ng, c in counter.items():
        Nc_c = Nc.get(c, 0)
        Nc_cp1 = Nc.get(c+1, 0)
        if Nc_c > 0 and Nc_cp1 > 0:
            c_star = (c+1) * (Nc_cp1 / Nc_c)
        else:
            c_star = float(c)
        C_star[ng] = c_star
    Punseen = None
    if V_power_n is not None:
        unseen_count = max(V_power_n - N_distinct, 0)
        if unseen_count > 0 and N_distinct > 0:
            Punseen = (N1 / N_distinct) / unseen_count
        else:
            Punseen = 0.0
    diagnostics = {
        "Nc": Nc,
        "N_distinct": N_distinct,
        "N1": N1,
        "max_c": max_c,
        "total_seen_tokens": sum(counter.values())
    }
    return C_star, Punseen, diagnostics

def normalize_joint_from_Cstar(C_star, punseen, V_power_n):
    """
    Convert C* to probabilities that sum to 1 by scaling seen mass and including unseen mass = punseen * unseen_count.
    Returns: P_seen (dict ngram->prob), p_unseen_each
    """
    sum_cstar = sum(C_star.values())
    unseen_count = max(V_power_n - len(C_star), 0)
    unseen_mass = (punseen * unseen_count) if punseen is not None else 0.0
    if sum_cstar == 0:
        scale = 0.0
    else:
        scale = max(0.0, (1.0 - unseen_mass) / sum_cstar)
    P_seen = {ng: cstar * scale for ng, cstar in C_star.items()}
    # final numeric adjust (small)
    total = sum(P_seen.values()) + (punseen or 0.0) * unseen_count
    if total > 0:
        # renormalize to exactly 1
        factor = 1.0 / total
        for ng in list(P_seen.keys()):
            P_seen[ng] *= factor
        if punseen is not None:
            punseen *= factor
    return P_seen, punseen

def save_pickle(obj, fn):
    with open(fn, "wb") as fh:
        pickle.dump(obj, fh)

def main():
    train = load_pickle(TRAIN_PICKLE)
    print(f"Loaded train ({len(train)} sentences). Building counts and Good-Turing models...")
    # compute vocab size from train tokens
    vocab = set()
    for s in train:
        vocab.update(s)
    vocab.update([SPECIAL_START, SPECIAL_END])
    V = len(vocab)
    for n in range(1, MAX_N+1):
        counts, total = build_counts(train, n)
        save_pickle(counts, f"counts_n{n}.pkl")
        Vpow = V ** n
        C_star, punseen, diag = good_turing_counts(counts, V_power_n=Vpow)
        P_seen, punseen = normalize_joint_from_Cstar(C_star, punseen, Vpow)
        save_pickle(P_seen, f"gt_joint_n{n}.pkl")
        with open(f"diagnostics_n{n}.json", "w", encoding="utf8") as fh:
            # JSON-friendly: reduce large Nc dict by converting keys to str
            dd = dict(diag)
            dd["Nc"] = {str(k): v for k, v in diag["Nc"].items()}
            dd["vocab_size"] = V
            fh.write(json.dumps(dd, indent=2))
        print(f"n={n}: distinct_seen={diag['N_distinct']} N1={diag['N1']} total_tokens={diag['total_seen_tokens']}")
    print("Done. Files produced: counts_n*.pkl, gt_joint_n*.pkl, diagnostics_n*.json")

if __name__ == "__main__":
    main()


Loaded train (142154 sentences). Building counts and Good-Turing models...
n=1: distinct_seen=90639 N1=45265 total_tokens=3151811
n=2: distinct_seen=893373 N1=635954 total_tokens=3151811
n=3: distinct_seen=1998504 N1=1725751 total_tokens=3151811
n=4: distinct_seen=2581780 N1=2415729 total_tokens=3151811
Done. Files produced: counts_n*.pkl, gt_joint_n*.pkl, diagnostics_n*.json


In [ ]:
#!/usr/bin/env python3
"""
Task 3: Compute sentence log-probabilities for validation and test sets using
the Good-Turing-smoothed joint models produced in task2.

Inputs:
 - val.pkl, test.pkl (from task1)
 - gt_joint_n{1..4}.pkl (from task2)
Outputs:
 - logprobs_n{1..4}_val.txt
 - logprobs_n{1..4}_test.txt
"""

import pickle, math
from collections import Counter
SPECIAL_START = "<s>"
SPECIAL_END = "</s>"
MAX_N = 4

def load_pickle(fn):
    with open(fn, "rb") as fh:
        return pickle.load(fh)

def sentence_ngrams(tokens, n):
    padded = [SPECIAL_START] * (n-1) + tokens + [SPECIAL_END]
    ngrams = [tuple(padded[i:i+n]) for i in range(len(padded)-n+1)]
    return ngrams

def conditional_prob(ngram, joint_n, joint_n_minus1, punseen_n, punseen_n_minus1):
    # joint_n: dict of joint probs for n-grams
    # joint_n_minus1: dict for (n-1)-grams
    if len(ngram) == 1:
        return joint_n.get(ngram, punseen_n if punseen_n is not None else 1e-16)
    joint = joint_n.get(ngram, punseen_n if punseen_n is not None else 1e-16)
    history = ngram[:-1]
    denom = joint_n_minus1.get(history, punseen_n_minus1 if punseen_n_minus1 is not None else 1e-12)
    if denom <= 0:
        denom = 1e-12
    p = joint / denom
    return max(p, 1e-16)

def sentence_logprob(tokens, order, joint_models):
    """
    joint_models: dict n -> (joint_probs_dict, punseen_each)
    """
    logp = 0.0
    n = order
    padded = [SPECIAL_START]*(n-1) + tokens + [SPECIAL_END]
    for i in range(n-1, len(padded)):
        ngram = tuple(padded[i-(n-1):i+1])
        if n == 1:
            joint_n, punseen_n = joint_models[1]
            p = joint_n.get(ngram, punseen_n if punseen_n is not None else 1e-16)
        else:
            joint_n, punseen_n = joint_models[n]
            joint_nm1, punseen_nm1 = joint_models[n-1]
            p = conditional_prob(ngram, joint_n, joint_nm1, punseen_n, punseen_nm1)
        logp += math.log(p)
    return logp

def main():
    val = load_pickle("val.pkl")
    test = load_pickle("test.pkl")
    joint_models = {}
    for n in range(1, MAX_N+1):
        jp = load_pickle(f"gt_joint_n{n}.pkl")  # dict ngram->prob
        # We didn't explicitly save punseen in task2's pickle. We'll approximate punseen by tiny value for safety.
        # If you saved punseen, load it; otherwise set None so code uses fallback.
        # Here: punseen unknown -> None
        joint_models[n] = (jp, None)

    for n in range(1, MAX_N+1):
        out_val = f"logprobs_n{n}_val.txt"
        out_test = f"logprobs_n{n}_test.txt"
        with open(out_val, "w", encoding="utf8") as fv:
            for sent in val:
                lp = sentence_logprob(sent, n, joint_models)
                fv.write(f"{lp}\n")
        with open(out_test, "w", encoding="utf8") as ft:
            for sent in test:
                lp = sentence_logprob(sent, n, joint_models)
                ft.write(f"{lp}\n")
        print(f"Wrote {out_val}, {out_test}")
    print("Done.")

if __name__ == "__main__":
    main()


Wrote logprobs_n1_val.txt, logprobs_n1_test.txt
Wrote logprobs_n2_val.txt, logprobs_n2_test.txt
Wrote logprobs_n3_val.txt, logprobs_n3_test.txt
Wrote logprobs_n4_val.txt, logprobs_n4_test.txt
Done.


In [ ]:
#!/usr/bin/env python3
"""
Task 4: Deleted-interpolated smoothing for quadrigram model.
We use the validation set to find interpolation weights λ1..λ4 (sum to 1)
to maximize log-likelihood of val data:
 P_interp(w | history) = sum_{k=1..4} λ_k * P_k(w | history_k)

Approach: coarse grid search on the 4-simplex (lambda1,lambda2,lambda3,lambda4).
Inputs required:
 - gt_joint_n1..n4 pickles (produced in task2)
 - val.pkl (produced in task1)
Outputs:
 - best_lambdas.txt
"""

import pickle, math
from itertools import product
SPECIAL_START = "<s>"
SPECIAL_END = "</s>"
MAX_N = 4

GRID_STEP = 0.1   # default coarse; lower -> finer but slower

def load_pickle(fn):
    with open(fn, "rb") as fh:
        return pickle.load(fh)

def conditional_from_joint(ngram, joint_n, joint_nm1, punseen_n=None, punseen_nm1=None):
    if len(ngram) == 1:
        return joint_n.get(ngram, punseen_n if punseen_n is not None else 1e-16)
    joint = joint_n.get(ngram, punseen_n if punseen_n is not None else 1e-16)
    history = ngram[:-1]
    denom = joint_nm1.get(history, punseen_nm1 if punseen_nm1 is not None else 1e-12)
    if denom <= 0:
        denom = 1e-12
    return max(joint / denom, 1e-16)

def sentence_ngrams(tokens, n):
    padded = [SPECIAL_START]*(n-1) + tokens + [SPECIAL_END]
    for i in range(n-1, len(padded)):
        yield tuple(padded[i-(n-1):i+1])

def load_joint_models():
    jm = {}
    for n in range(1, MAX_N+1):
        jp = load_pickle(f"gt_joint_n{n}.pkl")
        jm[n] = (jp, None)   # None punseen; if you have it, include
    return jm

def compute_val_loglik(lambdas, val_sents, joint_models):
    total_loglik = 0.0
    for s in val_sents:
        padded = [SPECIAL_START]*(MAX_N-1) + s + [SPECIAL_END]
        for i in range(MAX_N-1, len(padded)):
            w = padded[i]
            history = padded[i-(MAX_N-1):i]
            # gather P_k for k=1..4
            probs = []
            for k in range(1, MAX_N+1):
                if k == 1:
                    ngram = (w,)
                else:
                    hist_k = history[-(k-1):] if (k-1) > 0 else []
                    ngram = tuple(hist_k + [w])
                joint_k, punseen_k = joint_models[k]
                if k == 1:
                    p_k = joint_k.get(ngram, punseen_k if punseen_k is not None else 1e-16)
                else:
                    joint_km1, punseen_km1 = joint_models[k-1]
                    p_k = conditional_from_joint(ngram, joint_k, joint_km1, punseen_k, punseen_km1)
                probs.append(p_k)
            p_interp = sum(l * p for l, p in zip(lambdas, probs))
            if p_interp <= 0:
                p_interp = 1e-16
            total_loglik += math.log(p_interp)
    return total_loglik

def grid_simplex(step=GRID_STEP):
    steps = int(1.0/step) + 1
    lambdas = []
    for i1 in range(steps):
        l1 = i1*step
        for i2 in range(steps):
            l2 = i2*step
            for i3 in range(steps):
                l3 = i3*step
                s = l1 + l2 + l3
                if s > 1.0 + 1e-12:
                    continue
                l4 = 1.0 - s
                if l4 < -1e-12:
                    continue
                lambdas.append((l1, l2, l3, l4))
    return lambdas

def main():
    val = load_pickle("val.pkl")
    joint_models = load_joint_models()
    best = (None, -1e300)
    grid = grid_simplex(GRID_STEP)
    print(f"Evaluating {len(grid)} lambda candidates (step={GRID_STEP}) on {len(val)} validation sentences...")
    for lamb in grid:
        ll = compute_val_loglik(lamb, val, joint_models)
        if ll > best[1]:
            best = (lamb, ll)
    print("Best lambdas:", best[0], "loglik:", best[1])
    with open("best_quadrigram_lambdas.txt", "w", encoding="utf8") as fh:
        fh.write(f"{best[0]}\nloglik={best[1]}\n")
    print("Saved best_quadrigram_lambdas.txt")

if __name__ == "__main__":
    main()


Evaluating 286 lambda candidates (step=0.1) on 1000 validation sentences...
Best lambdas: (0.2, 0.5, 0.2, 0.10000000000000009) loglik: -113867.8890612377
Saved best_quadrigram_lambdas.txt
